In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline


<!---
Latex Macros
-->
$$
\newcommand{\Xs}{\mathcal{X}}
\newcommand{\Ys}{\mathcal{Y}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\weights}{\mathbf{w}}
\newcommand{\balpha}{\boldsymbol{\alpha}}
\newcommand{\bbeta}{\boldsymbol{\beta}}
\newcommand{\aligns}{\mathbf{a}}
\newcommand{\align}{a}
\newcommand{\source}{\mathbf{s}}
\newcommand{\target}{\mathbf{t}}
\newcommand{\ssource}{s}
\newcommand{\starget}{t}
\newcommand{\repr}{\mathbf{f}}
\newcommand{\repry}{\mathbf{g}}
\newcommand{\bar}{\,|\,}
\newcommand{\x}{\mathbf{x}}
\newcommand{\prob}{p}
\newcommand{\Pulp}{\text{Pulp}}
\newcommand{\Fiction}{\text{Fiction}}
\newcommand{\PulpFiction}{\text{Pulp Fiction}}
\newcommand{\pnb}{\prob^{\text{NB}}}
\newcommand{\vocab}{V}
\newcommand{\params}{\boldsymbol{\theta}}
\newcommand{\param}{\theta}
\DeclareMathOperator{\perplexity}{PP}
\DeclareMathOperator{\argmax}{argmax}
\DeclareMathOperator{\argmin}{argmin}
\newcommand{\train}{\mathcal{D}}
\newcommand{\counts}[2]{\#_{#1}(#2) }
\newcommand{\length}[1]{\text{length}(#1) }
\newcommand{\indi}{\mathbb{I}}
$$

In [2]:
%%HTML
<style>
td,th {
    font-size: x-large;
    text-align: left;
}
</style>

# Representations / Vector Semantics 

After this lecture you should:
* know why we go from n-hot vectors (sparse) to embedding inputs (dense inputs)
* understand various ways how to obtain word embeddings

### Recap: Feed-forward Neural Network


$$NN_{MLP1}(\mathbf{x})=g(\mathbf{xW^1+b^1})\mathbf{W^2}+\mathbf{b^2}$$

<img src="pics/nn.png">


<img src="pics/yg-compgraph1.png">


What is the input $\textbf{x}$?

### What makes language so challenging (continued)



<img src="pics/Cute_grey_kitten.jpg" width="550px"> <!-- kitten vs kat -->



Commonest linguistic way of thinking of meaning:
language *forms come with meaning* (arbitrary connection)

 signifier (symbol) <=> signified (idea of thing) = denotational semantics 
 (e.g., Ferdinand de Saussure)
<img src="pics/saussure.png" width="600px">

# Representations



### Lecture outline

* **Representations of words**
     * the difference between **sparse discrete** (one-hot/sparse binary/sparse count-based/n-hot) and 
     * **dense continuous** feature representations
* How to acquire **word representations** / **distributional similarity** / vector semantics
    * Traditional (LSA) vs
    * Neural (aka. embeddings, word2vec)
* **CBOW** classifier


## How do we represent the meaning of a word?

Definition: **meaning** (Webster dictionary)

- the idea that is represented by a word, phrase etc.
- the idea that a person wants to express using words, signs, etc
- the idea that is expressed in a work of writing, art, etc



<img src="pics/cm1.png" width="500px">
(*Slides by C.Manning*)

### Problems with resources like WordNet

* Great resource but missing **nuance** (e.g., 'proficient' is a synonym for 'good'. It is only correct in some contexts)
* Missing new meanings of words (very hard to keep up-to-date!)
* Subjective
* Requires human labor to create and maintain

## A solution via Distributional Hypothesis

Formulated in the 50s by linguists  (e.g., Harris 1954, Firth 1957)

<center>**"You shall know a word by the company it keeps"** (Firth, J. R. 1957:11)</center>

<img src="pics/flødebolle.png">

### "The company it keeps": Representing words by their context

* **Core idea:** The meaning of a word is represented by the words frequently occur close-by
* One of the most successful ideas in statistical NLP
* Word $w$, *context* around $w$ (typically a fixed-size window)

<img src="pics/cm5.png" width="550px">

## Why talk about representations? ##

* Machine Learning, features are representations
* Better representations, better performance
* Representation Learning ("Deep Learning"), trendy

(*some slides adapted from S.Riedel*)

## What makes a good representation? ##

1. Representations are **distinct**
2. **Similar** words have **similar** representations

## Sparse Binary 

So far we used **sparse** inputs (n-hot encodings) a.k.a. **discrete representations**. In fact, the vast majority of (rule-based and statistical) NLP work regarded words as atomic symbols:

**sparse binary / discrete representation**: a vector with one 1 and a lot of zeroes (one-hot). The dimensionality is determined by $|V|$, the vocabulary size.


## Sparse Binary Example ##

sb = sparse binary 

* $\mathbb{V} = \{\textrm{cat}, \textrm{dog}, \textrm{table}\}$
* $f_{sb}(\textrm{cat}) = [1, 0, 0]$
* $f_{sb}(\textrm{dog}) = [0, 1, 0]$
* $f_{sb}(\textrm{table}) = [0, 0, 1]$

## Sparse Binary Representations Visualised ##

<img src="pics/sparse_vec.png" width=350>


**Similarity** on discrete representations? 

E.g., Hamming distance:
$$\mathbf{x}_{cat} \wedge \mathbf{x}_{dog} = 0$$

But our vectors are orthogonal. There	is	no	natural	notion	of	similarity	in	a	set	of one-hot	vectors.

<img src="pics/cm4.png" width="450px">

## Cosine Similarity ##

* $cos(u, v) = \frac{u \cdot v}{||u|| ||v||}$
* $cos(u, v) \mapsto [-1, 1]$
* $cos(u, v) = 1$; identical
* $cos(u, v) = -1$; opposites
* $cos(u, v) = 0$; orthogonal

Note the different formulation in SciPy: $cos(u, v) = 1 - \frac{u \cdot v}{||u|| ||v||}$

## Cosine Similarity Visualised ##

<center><img src="http://blog.christianperone.com/wp-content/uploads/2013/09/cosinesimilarityfq1.png" width="110%"></center>

http://blog.christianperone.com/2013/09/machine-learning-cosine-similarity-for-vector-space-models-part-iii/

## Sparse Binary Similarities ##

* $cos(f_{sb}(\textrm{cat}), f_{sb}(\textrm{dog})) = 0$
* $cos(f_{sb}(\textrm{cat}), f_{sb}(\textrm{table})) = 0$
* $cos(f_{sb}(\textrm{table}), f_{sb}(\textrm{dog})) = 0 $

## From sparse binary to dense continuous representations

Probably the biggest jump when moving from traditional linear models with sparse inputs to deep neural networks is to stop representing each feature as a unique dimension, but instead represent them as **dense vectors** (Goldberg, 2015).

## Dense Continuous Representations ##

## Dense Continuous Representations ##

* $f_{dc}(w) \mapsto \mathbb{R}^d$
* "Embed" words as matrix rows
* Dimensionality: $d$ (hyperparameter)
* Word embedding matrix: $W \in \mathbb{R}^{|\mathbb{V}| \times d}$



## Dense Continuous Example ##

dc = dense continuous 

* $\mathbb{V} = \{\textrm{cat}, \textrm{dog}, \textrm{table}\}$
* $d = 2$
* $W \in \mathbb{R}^{3 \times 2}$

where:
* $f_{sb}(\textrm{cat}) = [0.7,0.8]$
* $f_{sb}(\textrm{dog}) = [0.75,0.6]$
* $f_{sb}(\textrm{table}) = [0.1,0.15]$


## Dense Continuous Representations Visualised ##

<img src="pics/dense_vec.png" width=350>


### Word vectors
Instead of using discrete representations, we will **embed** words into a high-dimensional feature space and represent each word by a lower-dimensional dense *vector* (aka. **word vector**), chosen such that its representation is close to vectors of words that appear in similar contexts.


### Word vectors

Note: **word vectors** are sometimes called **word embeddings** or **word representations**. They are a **distributed representation**.

<img src="pics/wordvector.png" width=300>


<img src="pics/cm7.png" width="700px">

### "The company it keeps" 

Core idea: learn the meaning of a word by looking at lots of lots of text (*unsupervised learning*) 

Core assumption: *similar words tend to occur in similar contexts*

<img src="pics/kitty-context.png">

* Traditionally: decompose a word co-occurence matrix (vector space models, distributional semantics)
* Neural world: a simple idea that works very well

## Method 1 - Predict! (aka Prediction-based methods)


* **Idea**: directly learn word vectors, i.e., **predict** the words with a neural network
* We are going to look at one very popular method, **word2vec** (Mikolov et al., 2013)

### Main idea of prediction-based approaches

* instead of capturing co-occurence statistics of words
* **predict context** (surrounding words of every word); in particular, predict words in a window of length $m$ around current word

* Most prominent approach: [word2vec](https://github.com/tmikolov/word2vec) by [Mikolov et al., 2013](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)



## Word2vec
<img src="pics/cmir1.png" width="600px">

<img src="pics/cmir2.png" width="600px">

<img src="pics/cm8.png">

<img src="pics/cm9.png" width="550px">

<img src="pics/cm10.png" width="550px">

Note: only one probability distribution, that of the output word appearing close to the center word

### Details of word2vec

* Given a large corpus of text
* Go	through	each position	$t$ in	the	text,	which	has	a	center	word	$c$ and	context	(“outside”)	words	$o$
* Use	the	similarity	of	the	word	vectors	for	$c$	and	$o$ to	calculate	the	probability	of	o given	c	



<img src="pics/cm11.png" width="550px">

$\theta$ is the vector representation of the words

<img src="pics/cm12.png" width="550px">

### Dot product

- dot product is a kind of similarity function: bigger if $u$ and $v$ more similar
- softmax to put them into a probability distribution

<img src="pics/cm13.png" width="550px">

<img src="pics/cm14.png" width="550px">

<img src="pics/cm15.png" width="550px">

<img src="pics/manning.png" width="550px">

How to calculate $P(w_{t+j}|w_t; \theta)$? 

Answer: use two vectors per word w: 
$o$ when $w$ is the outside word (context), and $c$ when $w$ is the current center word; 

Then, the probability of a word in the context ($o$) given the current word $c$ is:

$$p(o|c) = \frac{exp(u_o^T v_c)}{\sum_{w=1}^W exp(u_w^T v_c)}$$

* Dot	product	compares	similarity	of	o and	c. Larger	dot	product	=	larger	probability
* denominator normalizes over entire vocabulary. What could be a problem?


For more details, see http://web.stanford.edu/class/cs224n/lectures/lecture2.pdf and chapter 6 of the SLP book https://web.stanford.edu/~jurafsky/slp3/6.pdf

<img src="http://www.gabormelli.com/RKB/images/a/a6/skip-gram_NNLM_architecture.150216.jpg">

NB. denominator $\sum$ over all words! In practice, *negative sampling* is used (randomly choose a word which is not in context as a negative sample)

<img src="pics/cmir3.png" width="600px">

<img src="pics/cmir4.png" width="600px">

### Many new ways to obtain embeddings!

A (biased) selection:

* **[FastText](https://arxiv.org/abs/1607.04606)** embeddings (Bojanowski et al., 2016): from subword characters (vectors are built from vectors of substrings of characters contained in it)
* **[ELMo](https://arxiv.org/pdf/1802.05365.pdf)** (Peters et al., 2018): deep *contextualized* word representation

## Method 2 (traditional): Count! (aka Count-based methods)

We can represent the "company" of a word in terms of a **word context (word co-occurence) matrix**. On the rows we have the words, on the columns their context.

**Contexts** can be of different types, for example:
* entire documents
* paragraphs
* a window around the word


Getting word vectors of by count-based method:

* Create the word-context matrix (count how often word appears in context)
* Maybe weight the counts (e.g., PMI or tf-idf
* Typically reduce dimensions using SVD

What you get: a vector representation of each word; can measure the closeness of words in this resulting *word vector space*

### Co-occurence matrix

* **dimensionality**: number of words $|V|$ (size of vocabulary) times number of documents (typically number of documents is huge)
* we want to **reduce** its dimensionality

### LSA - Latent Semantic Analysis (Singular Value Decomposition - SVD)

Approximate a matrix $\mathbf{C}$ through a decomposition into three submatrices (**of smaller dimensionality**):

$$\mathbf{C} \approx \mathbf{U \sum V^T}$$




<img src="https://simonpaarlberg.com/posts/2012-06-28-latent-semantic-analyses/box2.png">

NB. $=$ should be $\approx$

* **Problem** with count-based approach: SVD computation cost scales quadratically with size of co-occurence matrix


The **traditional** approach for extracting features for an NLP model is:

* extract a set of core linguistic features $f_1,..f_n$
* define a vector whose length is the total number of features; (n-hot): 1 at position k if the k-th feature is active; this feature vector represents the **instance** $\mathbf{x}$  (**sparse representation**, n-hot encoding)
* use $\mathbf{x}$ as representation for an instance, train the model


## Traditional way: Representing text as BOW (sparse discrete)

<img src="pics/bow2.png" width="550px">

Instead, in a neural approach it is typical to:

* extract a set of core linguistic features $f_1,..f_n$
* define a **vector** for **each feature** (lookup Embedding table)
* **combine** vectors of features to get the vector representation for the **instance** $\mathbf{x}$ (**dense representation**)
* use $\mathbf{x}$ as representation for an instance, train the model


    

## Computational Graph

$$NN_{MLP1}(\mathbf{x})=g(\mathbf{xW^1+b^1})\mathbf{W^2}+\mathbf{b^2}$$

<img src="pics/yg-compgraph1.png">



Computational Graph with input:
<img src="pics/yg-compgraph2.png">

The values of the *embedding vectors* (values of the vectors in Fig 1 b)) are treated as model parameters and **trained together** with the other parameters of the model (weights).

Unrolled (graph with concrete input, expected output, and loss node, Goldberg Figure 3 c):
<img src="pics/yg-compgraph3.png">

### CBOW model

A simple classification model that uses embeddings as representation is the CBOW model: it uses the sum (or average) of the embeddings of the words in the sentence and often works surprisingly well.

$$ \mbox{CBOW}(w_i,..,w_n) = \sum_i^n E[w_i] $$



<img src="pics/gn-cbow.png" width="550px">


## Making it deeper

<img src="pics/deepbow.png">
[Image credit: DyNet tutorial] 


### Tu sum up: Word Embeddings

So, in deep learning approaches to NLP words are represented as dense vectors. Where do these word vectors (embeddings) come from?

* **off-the-shelf embeddings**: you can also use trained, available embeddings (e.g. estimated with *word2vec*) and *initialize* the embedding layer of the network with your pretrained (unsupervised) word embeddings
* **task-specific embeddings**: you could also train your embeddings from scratch with the data for your task. In this case, the vectors are typically **randomly initialized** (small numbers around 0) and *trained with the network*. At the end you can read them off the network.

Remember, today we have seen tree ways to get embeddings:

1. Traditional methods (also called 'count' methods): SVD on a co-occurence matrix (=LSA)
2. Neural method #1 (also called 'predict' methods): word2vec (train on large unlabeled corpus)
3. Neural method #2 (also a 'predict' method, but task-specific): train your embeddings on your supervised task, read them off at the end (typically less used as you will have less supervised training data, it's easier to get loads of unlabeled text)

### Inputs of different lengths

In our classification example today we have one simplification: the input is always of the same size (namely, n words, a fixed window). 

However, in NLP we typically never have fixed size inputs, sentences are of different length. The neural network however needs inputs of fixed size. So how to deal with it?

That's for the next lecture.


### References

* Yoav Goldberg's primer chapter 3-5: [A Primer on Neural Network Models for Natural Language Processing](http://arxiv.org/abs/1510.00726)
* Simon Paarlberg's [blog on LSA](https://simonpaarlberg.com/post/latent-semantic-analyses/)
* Richard Socher's [lecture 2](https://www.youtube.com/watch?v=xhHOL3TNyJs)
* Baroni et al., (2014) [Don't count, predict! A systematic comparison of context-counting vs. context-predicting semantic vectors](https://aclanthology.coli.uni-saarland.de/papers/P14-1023/p14-1023)
* Fokkens et al., (2013) [Offspring from Reproduction Problems:
What Replication Failure Teaches Us](http://aclweb.org/anthology/P/P13/P13-1166.pdf)
* Mikolov et al., (2013) [Distributed Representations of Words and Phrases
and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)